In [1]:
from datasets import load_dataset, DatasetDict

# Load from the specific directory
dataset = load_dataset("text", 
    data_dir="./datasets/arxiv-dataset",
    data_files={
        "train": "train.txt",
        "test": "test.txt",
        "validation": "val.txt"
    }
)


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/29 [00:00<?, ?it/s]

In [2]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("text", 
    data_files={
        "train": "/scratch/ag8172/NLP/Project/RMT/recurrent-memory-transformer/datasets/arxiv-dataset/train.txt",
        "test": "/scratch/ag8172/NLP/Project/RMT/recurrent-memory-transformer/datasets/arxiv-dataset/test.txt",
        "validation": "/scratch/ag8172/NLP/Project/RMT/recurrent-memory-transformer/datasets/arxiv-dataset/val.txt"
    }
    )

dataset = dataset.shuffle(seed=42)

train_dataset = dataset["train"].select(range(8500))  # First 8500 for training
val_dataset = dataset["train"].select(range(8500, 9000))  # Next 500 for validation
test_dataset = dataset["test"].select(range(1000))  # 1000 for testing (from test split)


Loading dataset shards:   0%|          | 0/29 [00:00<?, ?it/s]

In [4]:
print("First few examples from training set:")
# print(train_dataset[:3])  # This will show first 3 examples

# To get basic information about the dataset
print("\nDataset info:")
print(f"Number of training examples: {len(train_dataset)}")
print(f"Features available: {train_dataset.features}")

# To see an individual example in detail
print("\nDetailed look at first example:")
print(train_dataset[0])

First few examples from training set:

Dataset info:
Number of training examples: 8500
Features available: {'text': Value(dtype='string', id=None)}

Detailed look at first example:
{'text': '{"article_id": "1506.05133", "article_text": ["arp  220 is the nearest ( @xmath3 77  mpc ) example of an ultraluminous infrared galaxy ( ulirg ) that supports star formation at extreme levels .", "it contains two nuclei separated by 350  pc , both surrounded by massive discs of dense molecular gas ( e.g. , * ? ? ?", "* ; * ? ? ?", "* ; * ? ? ?", "* ; * ? ? ?", "* ; * ? ? ?", "radio detections of supernovae at a rate of 13 yr@xmath4 @xcite confirm that huge populations of massive stars are present with an implied star formation rate ( sfr ) of @xmath5  yr@xmath4 .", "although arp 220 could contain active galactic nuclei ( agns ) , particularly in the western nucleus , the observed supernova rates indicate that star formation provides a substantial fraction of the power radiated by the nuclei .", "th

In [9]:
import json

x = json.loads(train_dataset[0]['text'])
x.keys()

dict_keys(['article_id', 'article_text', 'abstract_text', 'labels', 'section_names', 'sections'])

In [22]:
len(x['abstract_text'])

5

## Run testing

In [5]:
from mem_size_ablation import *
from load_saved_model_ablation import *

In [65]:
# checkpoint_path = "saved_models/mem32_in4096_b1_e3_1733410694/checkpoint_epoch_2.pt"  # Adjust path as needed
    
# print("Loading model...")
# model, tokenizer, config, metrics = load_checkpoint(checkpoint_path)


from transformers import GPT2LMHeadModel, GPT2TokenizerFast

model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')

In [6]:
device = 'cuda'

In [7]:
from datasets import load_dataset
def prepare_dataset(tokenizer):
    # test = load_dataset("wikitext", "wikitext-2-v1", split="test")
    # test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
    # test = load_dataset("roneneldan/TinyStories", split="validation[:5000]")
    test = load_dataset("wikitext", "wikitext-103-v1", split="test")

    encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

    return encodings

encodings = prepare_dataset(tokenizer)

NameError: name 'tokenizer' is not defined

In [68]:
import torch
from tqdm import tqdm

# def run_test(max_length,stride,
max_length = 1024
stride = 1024
seq_len = encodings.input_ids.size(1)

nll_sum = 0.0
n_tokens = 0
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100
    # print(begin_loc,end_loc,trg_len)

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss

    # Accumulate the total negative log-likelihood and the total number of tokens
    num_valid_tokens = (target_ids != -100).sum().item()  # number of valid tokens in target_ids
    batch_size = target_ids.size(0)
    num_loss_tokens = num_valid_tokens - batch_size  # subtract batch_size due to internal label shift
    nll_sum += neg_log_likelihood * num_loss_tokens
    n_tokens += num_loss_tokens

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

avg_nll = nll_sum / n_tokens  # average negative log-likelihood per token
ppl = torch.exp(avg_nll)



  0%|          | 0/280 [00:00<?, ?it/s]

  1%|          | 3/280 [00:00<00:12, 21.43it/s]

  2%|▏         | 6/280 [00:00<00:11, 22.92it/s]

  3%|▎         | 9/280 [00:00<00:11, 23.87it/s]

  4%|▍         | 12/280 [00:00<00:10, 24.39it/s]

  5%|▌         | 15/280 [00:00<00:10, 24.53it/s]

  6%|▋         | 18/280 [00:00<00:10, 24.76it/s]

  8%|▊         | 21/280 [00:00<00:10, 24.91it/s]

  9%|▊         | 24/280 [00:00<00:10, 24.94it/s]

 10%|▉         | 27/280 [00:01<00:10, 25.03it/s]

 11%|█         | 30/280 [00:01<00:09, 25.09it/s]

 12%|█▏        | 33/280 [00:01<00:09, 25.10it/s]

 13%|█▎        | 36/280 [00:01<00:09, 25.14it/s]

 14%|█▍        | 39/280 [00:01<00:09, 25.18it/s]

 15%|█▌        | 42/280 [00:01<00:09, 25.04it/s]

 16%|█▌        | 45/280 [00:01<00:09, 25.07it/s]

 17%|█▋        | 48/280 [00:01<00:09, 25.15it/s]

 18%|█▊        | 51/280 [00:02<00:09, 25.09it/s]

 19%|█▉        | 54/280 [00:02<00:08, 25.15it/s]

 20%|██        | 57/280 [00:02<00:08, 25.20it/s]

 21%|██▏  

In [69]:
ppl

tensor(29.1627, device='cuda:0')

In [64]:
ppl

tensor(29.1627, device='cuda:0')

In [5]:
def run_perplexity_test(encodings, max_length=1024, stride=1024, n_chunks=4):
    seq_len = encodings.input_ids.size(1)
    nll_sum = 0.0
    n_tokens = 0
    
    # Process text in chunks of max_length
    for chunk in range(n_chunks):
        chunk_start = chunk * max_length
        chunk_end = min((chunk + 1) * max_length, seq_len)
        
        prev_end_loc = chunk_start
        # Process each chunk with sliding windows
        for begin_loc in range(chunk_start, chunk_end, stride):
            end_loc = min(begin_loc + max_length, chunk_end)
            trg_len = end_loc - prev_end_loc
            
            input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
            target_ids = input_ids.clone()
            target_ids[:, :-trg_len] = -100
            
            with torch.no_grad():
                outputs = model(input_ids, labels=target_ids)
                neg_log_likelihood = outputs.loss
                
            num_valid_tokens = (target_ids != -100).sum().item()
            batch_size = target_ids.size(0)
            num_loss_tokens = num_valid_tokens - batch_size
            
            nll_sum += neg_log_likelihood * num_loss_tokens
            n_tokens += num_loss_tokens
            prev_end_loc = end_loc
            
    avg_nll = nll_sum / n_tokens
    ppl = torch.exp(avg_nll)
    return ppl



In [9]:
ppl_1024_chunks = run_perplexity_test(encodings,max_length=1024, stride=1024, n_chunks=4)
ppl_1024_chunks

../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [11,0,0], thread: [64,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [11,0,0], thread: [65,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [11,0,0], thread: [66,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [11,0,0], thread: [67,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [11,0,0], thread: [68,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [11,0,0], thread: [69,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [11,0,0], t

RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [11]:
ppl_4096 = run_perplexity_test(encodings, max_length=4096, stride=4096, n_chunks=1)
ppl_4096

tensor(37.9856, device='cuda:0')

## Run a Script for all 

In [5]:
from datasets import load_dataset
def prepare_dataset(tokenizer):
    # test = load_dataset("wikitext", "wikitext-2-v1", split="test")
    test = test = load_dataset("roneneldan/TinyStories", split="validation[:5000]")
    encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

    return encodings

In [6]:
def run_perplexity_test(encodings, max_length=1024, stride=1024, n_chunks=4):
    seq_len = encodings.input_ids.size(1)
    nll_sum = 0.0
    n_tokens = 0
    
    # Process text in chunks of max_length
    for chunk in range(n_chunks):
        chunk_start = chunk * max_length
        chunk_end = min((chunk + 1) * max_length, seq_len)
        
        prev_end_loc = chunk_start
        # Process each chunk with sliding windows
        for begin_loc in range(chunk_start, chunk_end, stride):
            end_loc = min(begin_loc + max_length, chunk_end)
            trg_len = end_loc - prev_end_loc
            
            input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
            target_ids = input_ids.clone()
            target_ids[:, :-trg_len] = -100
            
            with torch.no_grad():
                outputs = model(input_ids, labels=target_ids)
                neg_log_likelihood = outputs.loss
                
            num_valid_tokens = (target_ids != -100).sum().item()
            batch_size = target_ids.size(0)
            num_loss_tokens = num_valid_tokens - batch_size
            
            nll_sum += neg_log_likelihood * num_loss_tokens
            n_tokens += num_loss_tokens
            prev_end_loc = end_loc
            
    avg_nll = nll_sum / n_tokens
    ppl = torch.exp(avg_nll)
    return ppl



In [13]:
    
import torch
from tqdm import tqdm
runs = [
"mem2_in4096_b1_e3_1733388505",
"mem4_in4096_b1_e3_1733394016",
"mem8_in4096_b1_e3_1733399520",
"mem16_in4096_b1_e3_1733405065",
"mem32_in4096_b1_e3_1733410694",
"mem64_in4096_b1_e3_1733416296",
]

for run in runs:
    checkpoint_path = f"saved_models/{run}/checkpoint_epoch_2.pt"  # Adjust path as needed
        
    print(f"Model {run.split('_')[0]}")
    model, tokenizer, config, metrics = load_checkpoint(checkpoint_path)
    encodings = prepare_dataset(tokenizer)
    
    

    
    # def run_test(max_length,stride,
    max_length = 4096
    stride = 4096
    seq_len = encodings.input_ids.size(1)
    
    nll_sum = 0.0
    n_tokens = 0
    prev_end_loc = 0
    for begin_loc in tqdm(range(0, seq_len, stride)):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100
        # print(begin_loc,end_loc,trg_len)
    
        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
    
            # loss is calculated using CrossEntropyLoss which averages over valid labels
            # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
            # to the left by 1.
            neg_log_likelihood = outputs.loss
    
        # Accumulate the total negative log-likelihood and the total number of tokens
        num_valid_tokens = (target_ids != -100).sum().item()  # number of valid tokens in target_ids
        batch_size = target_ids.size(0)
        num_loss_tokens = num_valid_tokens - batch_size  # subtract batch_size due to internal label shift
        nll_sum += neg_log_likelihood * num_loss_tokens
        n_tokens += num_loss_tokens
    
        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    avg_nll = nll_sum / n_tokens  # average negative log-likelihood per token
    ppl = torch.exp(avg_nll)
        
    # ppl_4096 = run_perplexity_test(encodings, max_length=4096, stride=4096, n_chunks=1)
    
    print(f"Context Window PPL: {ppl}")
    # print(f"Full PPL: {ppl_4096}")
    print('--------------------')

Model mem2


/scratch/ag8172/NLP/Project/RMT/recurrent-memory-transformer/load_saved_model_ablation.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoin

Running with config: {'input_size': 4096, 'memory_size': 2, 'batch_size': 1, 'num_epochs': 3, 'model_name': 'gpt2', 'block_size': 1020, 'n_segments': 5, 'history_size': 4080, 'learning_rate': 0.0001, 'train_steps': 100, 'eval_steps': 100, 'run_name': 'mem2_in4096_b1_e3_1733427878', 'save_dir': PosixPath('saved_models/mem2_in4096_b1_e3_1733427878')}


Token indices sequence length is longer than the specified maximum sequence length for this model (1005236 > 1024). Running this sequence through the model will result in indexing errors
100%|█████████▉| 245/246 [00:43<00:00,  5.63it/s]


Context Window PPL: 45.756961822509766
--------------------
Model mem4
Running with config: {'input_size': 4096, 'memory_size': 4, 'batch_size': 1, 'num_epochs': 3, 'model_name': 'gpt2', 'block_size': 1016, 'n_segments': 5, 'history_size': 4064, 'learning_rate': 0.0001, 'train_steps': 100, 'eval_steps': 100, 'run_name': 'mem4_in4096_b1_e3_1733427934', 'save_dir': PosixPath('saved_models/mem4_in4096_b1_e3_1733427934')}


...

Context Window PPL: 45.806922912597656
--------------------
Model mem32
Running with config: {'input_size': 4096, 'memory_size': 32, 'batch_size': 1, 'num_epochs': 3, 'model_name': 'gpt2', 'block_size': 960, 'n_segments': 5, 'history_size': 3840, 'learning_rate': 0.0001, 'train_steps': 100, 'eval_steps': 100, 'run_name': 'mem32_in4096_b1_e3_1733428101', 'save_dir': PosixPath('saved_models/mem32_in4096_b1_e3_1733428101')}


Token indices sequence length is longer than the specified maximum sequence length for this model (1005236 > 1024). Running this sequence through the model will result in indexing errors
100%|█████████▉| 245/246 [00:45<00:00,  5.35it/s]


Context Window PPL: 49.698638916015625
--------------------
Model mem64
Running with config: {'input_size': 4096, 'memory_size': 64, 'batch_size': 1, 'num_epochs': 3, 'model_name': 'gpt2', 'block_size': 896, 'n_segments': 5, 'history_size': 3584, 'learning_rate': 0.0001, 'train_steps': 100, 'eval_steps': 100, 'run_name': 'mem64_in4096_b1_e3_1733428158', 'save_dir': PosixPath('saved_models/mem64_in4096_b1_e3_1733428158')}


Token indices sequence length is longer than the specified maximum sequence length for this model (1005236 > 1024). Running this sequence through the model will result in indexing errors
100%|█████████▉| 245/246 [00:47<00:00,  5.15it/s]

Context Window PPL: 55.40236282348633
--------------------


In [8]:
import torch
from tqdm import tqdm

def calculate_perplexity(model, encodings, model_max_length, device):
    seq_len = encodings.input_ids.size(1)
    nll_sum = 0.0
    n_tokens = 0
    prev_end_loc = 0

    # If sequence length is greater than model's max length, process in chunks
    if seq_len > model_max_length:
        print('here')
        stride = model_max_length  # No overlap when processing long sequences
        
        # Process in chunks of model_max_length
        for begin_loc in tqdm(range(0, seq_len, stride)):
            end_loc = min(begin_loc + model_max_length, seq_len)
            trg_len = end_loc - prev_end_loc
            
            input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
            target_ids = input_ids.clone()
            target_ids[:, :-trg_len] = -100
            
            with torch.no_grad():
                outputs = model(input_ids, labels=target_ids)
                neg_log_likelihood = outputs.loss
            
            num_valid_tokens = (target_ids != -100).sum().item()
            batch_size = target_ids.size(0)
            num_loss_tokens = num_valid_tokens - batch_size
            
            nll_sum += neg_log_likelihood * num_loss_tokens
            n_tokens += num_loss_tokens
            prev_end_loc = end_loc
            
            if end_loc == seq_len:
                break
    else:
        # If sequence fits in model's context window, process it all at once
        input_ids = encodings.input_ids.to(device)
        target_ids = input_ids.clone()
        
        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss
        
        num_valid_tokens = (target_ids != -100).sum().item()
        batch_size = target_ids.size(0)
        num_loss_tokens = num_valid_tokens - batch_size
        
        nll_sum += neg_log_likelihood * num_loss_tokens
        n_tokens += num_loss_tokens

    avg_nll = nll_sum / n_tokens
    ppl = torch.exp(avg_nll)
    return ppl

# Usage example:
# For 1024 model
ppl_1024 = calculate_perplexity(model, encodings, model_max_length=1024, device=device)
print(ppl_1024)
# For 4096 model
# ppl_4096 = calculate_perplexity(model_4096, encodings, model_max_length=4096, device=device)

here


100%|█████████▉| 280/281 [00:11<00:00, 25.12it/s]

tensor(29.9405, device='cuda:0')


In [19]:
ppl_4096 = calculate_perplexity(model_4096, encodings, model_max_length=1024, device=device)

NameError: name 'model_4096' is not defined

In [22]:
4 * 256 *256, 1024 * 1024

(262144, 1048576)

## Run Older Models

In [16]:
from datasets import load_dataset
def prepare_dataset(tokenizer):
    test = load_dataset("wikitext", "wikitext-2-v1", split="test")
    # test = test = load_dataset("roneneldan/TinyStories", split="validation[:5000]")
    # test = load_dataset("pg19", split="test[:100]")
    # test = test.select(range(200))
    # test = load_dataset("wikitext", "wikitext-103-v1", split="test")
    # test = load_dataset("wikipedia", "20220301.en", split="train[:1%]")
    
    encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

    return encodings

In [17]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from modeling_rmt.language_modeling import MemoryCell, RecurrentWrapper
import argparse

def load_rmt_model(model_path, args=None):
    """
    Load a saved RMT model and its tokenizer.
    
    Args:
        model_path (str): Path to the saved model (.bin file)
        args (argparse.Namespace, optional): Arguments containing model configuration.
                                          If None, default values will be used.
    
    Returns:
        tuple: (loaded_model, tokenizer)
    """
    # Default arguments if none provided
    if args is None:
        class DefaultArgs:
            def __init__(self):
                self.model_name = 'gpt2'
                self.memory_size = 4
                self.input_size = 1024
                self.n_segments = 2
                self.lora_r = 16
                self.lora_alpha = 32
                self.lora_dropout = 0.1
        args = DefaultArgs()

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(args.model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Initialize base model
    base_model = AutoModelForCausalLM.from_pretrained(args.model_name)
    
    # Configure LoRA
    lora_config = LoraConfig(
        r=args.lora_r,
        lora_alpha=args.lora_alpha,
        target_modules=["c_attn"],
        lora_dropout=args.lora_dropout,
        bias="none",
        task_type="CAUSAL_LM"
    )
    
    # Prepare model with LoRA
    model = prepare_model_for_kbit_training(base_model)
    model = get_peft_model(model, lora_config)
    
    # Set up RMT wrapper
    block_size = 1024 - 2 * args.memory_size
    cell = MemoryCell(model, num_mem_tokens=args.memory_size)
    rmt_model = RecurrentWrapper(cell,
                               segment_size=block_size,
                               max_n_segments=args.n_segments)
    
    # Load saved weights
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    state_dict = torch.load(model_path, map_location=device)
    rmt_model.load_state_dict(state_dict)
    rmt_model = rmt_model.to(device)
    
    return rmt_model, tokenizer

def load_rmt_model_lora(model_path,mem_size, config=None):
    """
    Load a saved RMT model from a .bin file
    
    Args:
        model_path (str): Path to the saved model .bin file
        config (dict, optional): Configuration dictionary containing model parameters.
            If None, will attempt to parse from the model filename.
            Should include: memory_size, input_size
            
    Returns:
        model: Loaded RecurrentWrapper model
    """
    if config is None:
        # Try to parse config from filename
        # Expected format: rmt_modelname_inputsize_memsize.bin
        try:
            filename = os.path.basename(model_path)
            parts = filename.replace('.bin', '').split('_')
            config = {
                'model_name': 'gpt2',
                'input_size': 4096,
                'memory_size': mem_size,
                'lora_r': 16,
                'lora_alpha': 32,
                'lora_dropout': 0.1
            }
        except:
            raise ValueError("Could not parse config from filename. Please provide config dictionary.")

    # Convert config to args namespace to match training setup
    args = argparse.Namespace(**config)
    
    # Setup model architecture
    base_model = AutoModelForCausalLM.from_pretrained(args.model_name)
    tokenizer = AutoTokenizer.from_pretrained(args.model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Setup LoRA configuration
    lora_config = LoraConfig(
        r=args.lora_r,
        lora_alpha=args.lora_alpha,
        target_modules=["c_attn"],
        lora_dropout=args.lora_dropout,
        bias="none",
        task_type="CAUSAL_LM"
    )
    
    # Prepare model with LoRA
    base_model = prepare_model_for_kbit_training(base_model)
    base_model = get_peft_model(base_model, lora_config)
    
    # Calculate segments and sizes
    block_size = 1024 - 2 * args.memory_size
    args.n_segments = math.ceil(args.input_size / block_size)
    args.history_size = (args.n_segments - 1) * block_size
    
    # Create RMT model
    cell = MemoryCell(base_model, num_mem_tokens=args.memory_size)
    model = RecurrentWrapper(cell,
                           segment_size=block_size,
                           max_n_segments=args.n_segments)
    
    # Load saved weights
    model.load_state_dict(torch.load(model_path))
    model.to('cuda')
    return model, tokenizer

In [18]:
# checkpoint_path = f"results/run_jnvnwhoq/model.bin"  # Adjust path as needed
        
# print(f"Model jnvnwhoq")
# model, tokenizer = load_rmt_model_lora(checkpoint_path,64)
# encodings = prepare_dataset(tokenizer)

In [19]:
import torch
from tqdm import tqdm
runs = [
"run_lvmpgbkg",
"run_0llr7i14",
"run_45a0rxa6",
]

runs = [
["run_hwr85mqs",128],
# ["run_jnvnwhoq",64],
# ["run_fla0gr8g",32],
# ["run_1vglsl0v",16],
# ["run_x1rqyscx",8],
# ["run_f6ez630v",4],
# ["run_wneyhtc1",2]
]

for run in runs:
    checkpoint_path = f"results/{run[0]}/model.bin"  # Adjust path as needed
        
    print(f"Model {run[0].split('_')[0]}")
    # model, tokenizer = load_rmt_model(checkpoint_path)
    model, tokenizer = load_rmt_model_lora(checkpoint_path,run[1])
    encodings = prepare_dataset(tokenizer)
    
    

    
    # def run_test(max_length,stride,
    # max_length = 1024
    # stride = 1024

    # lora memsize models
    max_length = 4096
    stride = 4096
    seq_len = encodings.input_ids.size(1)
    
    nll_sum = 0.0
    n_tokens = 0
    prev_end_loc = 0
    for begin_loc in range(0, seq_len, stride):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100
        # print(begin_loc,end_loc,trg_len)
    
        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
    
            # loss is calculated using CrossEntropyLoss which averages over valid labels
            # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
            # to the left by 1.
            neg_log_likelihood = outputs.loss
    
        # Accumulate the total negative log-likelihood and the total number of tokens
        num_valid_tokens = (target_ids != -100).sum().item()  # number of valid tokens in target_ids
        batch_size = target_ids.size(0)
        num_loss_tokens = num_valid_tokens - batch_size  # subtract batch_size due to internal label shift
        nll_sum += neg_log_likelihood * num_loss_tokens
        n_tokens += num_loss_tokens
    
        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    avg_nll = nll_sum / n_tokens  # average negative log-likelihood per token
    ppl = torch.exp(avg_nll)
    
    # ppl_4096 = run_perplexity_test(encodings, max_length=1024, stride=1024, n_chunks=1)
    
    print(f"Context Window PPL: {ppl}")
    # print(f"Full PPL: {ppl_4096}")
    print('--------------------')

Model run


/state/partition1/job-54559517/ipykernel_485046/1110075982.py:134: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))
Token indices 

Context Window PPL: 37.36375427246094
--------------------


In [7]:
checkpoint_path = "saved_models/mem32_in4096_b1_e3_1733410694/checkpoint_epoch_2.pt"  # Adjust path as needed
    
print("Loading model...")
model_4096, tokenizer_4096, config, metrics = load_checkpoint(checkpoint_path)


from transformers import GPT2LMHeadModel, GPT2TokenizerFast

model_1024 = GPT2LMHeadModel.from_pretrained('gpt2').to(device)
tokenizer_1024 = GPT2TokenizerFast.from_pretrained('gpt2')

Loading model...


/scratch/ag8172/NLP/Project/RMT/recurrent-memory-transformer/load_saved_model_ablation.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoin

Running with config: {'input_size': 4096, 'memory_size': 32, 'batch_size': 1, 'num_epochs': 3, 'model_name': 'gpt2', 'block_size': 960, 'n_segments': 5, 'history_size': 3840, 'learning_rate': 0.0001, 'train_steps': 100, 'eval_steps': 100, 'run_name': 'mem32_in4096_b1_e3_1733437824', 'save_dir': PosixPath('saved_models/mem32_in4096_b1_e3_1733437824')}


In [8]:
from datasets import load_dataset
def prepare_dataset(tokenizer):
    # test = load_dataset("wikitext", "wikitext-2-v1", split="test")
    test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
    # test = load_dataset("roneneldan/TinyStories", split="validation[:5000]")
    encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

    return encodings

encodings_4096 = prepare_dataset(tokenizer_4096)
encodings_1024 = prepare_dataset(tokenizer_1024)

Token indices sequence length is longer than the specified maximum sequence length for this model (287644 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (287644 > 1024). Running this sequence through the model will result in indexing errors


In [9]:
import torch
from tqdm import tqdm

def calculate_perplexity(model, encodings, max_length, stride, device):
    """
    Calculate perplexity for a given model and encodings, processing in chunks.
    
    Args:
        model: The language model
        encodings: The encoded text
        max_length: Maximum sequence length the model can handle
        stride: How much to shift the window by (usually equal to max_length for non-overlapping chunks)
        device: The device to run the model on
    """
    seq_len = encodings.input_ids.size(1)
    nll_sum = 0.0
    n_tokens = 0
    prev_end_loc = 0
    
    for begin_loc in tqdm(range(0, seq_len, stride)):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100
        
        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss
            
        num_valid_tokens = (target_ids != -100).sum().item()
        batch_size = target_ids.size(0)
        num_loss_tokens = num_valid_tokens - batch_size
        
        nll_sum += neg_log_likelihood * num_loss_tokens
        n_tokens += num_loss_tokens
        prev_end_loc = end_loc
        
        if end_loc == seq_len:
            break
            
    avg_nll = nll_sum / n_tokens
    ppl = torch.exp(avg_nll)
    
    return ppl, avg_nll, n_tokens

# Example usage for model with 1024 context window
def evaluate_1024_model(model_1024, encodings, device):
    """
    Evaluate model with 1024 context window on 4096 sequence
    """
    ppl, avg_nll, n_tokens = calculate_perplexity(
        model=model_1024,
        encodings=encodings,
        max_length=1024,
        stride=1024,  # non-overlapping chunks
        device=device
    )
    return {
        'perplexity': ppl,
        'avg_nll': avg_nll,
        'tokens_processed': n_tokens
    }

# Example usage for model with 4096 context window
def evaluate_4096_model(model_4096, encodings, device):
    """
    Evaluate model with 4096 context window on same sequence
    """
    ppl, avg_nll, n_tokens = calculate_perplexity(
        model=model_4096,
        encodings=encodings,
        max_length=4096,
        stride=4096,  # process entire sequence at once
        device=device
    )
    return {
        'perplexity': ppl,
        'avg_nll': avg_nll,
        'tokens_processed': n_tokens
    }

# Compare both models
def compare_models(model_1024, model_4096, encodings_1024, encodings_4096, device):
    """
    Compare perplexity between models with different context windows
    """
    results_1024 = evaluate_1024_model(model_1024, encodings, device)
    results_4096 = evaluate_4096_model(model_4096, encodings, device)
    
    print("1024 Context Window Results:")
    print(f"Perplexity: {results_1024['perplexity']:.2f}")
    print(f"Average NLL: {results_1024['avg_nll']:.4f}")
    print(f"Tokens processed: {results_1024['tokens_processed']}")
    print("\n4096 Context Window Results:")
    print(f"Perplexity: {results_4096['perplexity']:.2f}")
    print(f"Average NLL: {results_4096['avg_nll']:.4f}")
    print(f"Tokens processed: {results_4096['tokens_processed']}")

In [10]:
compare_models(model_1024, model_4096, encodings_1024, encodings_4096, device)

 99%|█████████▊| 70/71 [00:12<00:00,  5.47it/s]

1024 Context Window Results:
Perplexity: 29.94
Average NLL: 3.3992
Tokens processed: 287363

4096 Context Window Results:
Perplexity: 35.71
Average NLL: 3.5755
Tokens processed: 287573


In [ ]:
so i guess we can say that 4096 rmt perp isnt so bad